In [1]:
import pandas as pd
import numpy as np
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error


In [2]:
# Read in previously merged cat/quant data
test_merged = pd.read_csv('test_merged.csv')
train_merged = pd.read_csv('train_merged.csv')

# Read in connectomes
test_connectome = pd.read_csv('/Users/rubyc/Desktop/Datathon/WIDS_Datathon2025_Team/Archive/widsdatathon2025/TEST/TEST_FUNCTIONAL_CONNECTOME_MATRICES.csv')
train_connectome = pd.read_csv('/Users/rubyc/Desktop/Datathon/WIDS_Datathon2025_Team/Archive/widsdatathon2025/TRAIN_NEW/TRAIN_FUNCTIONAL_CONNECTOME_MATRICES_new_36P_Pearson.csv')

# Read in training solutions
y_df_raw = pd.read_excel('/Users/rubyc/Desktop/Datathon/WIDS_Datathon2025_Team/Archive/widsdatathon2025/TRAIN_NEW/TRAINING_SOLUTIONS.xlsx')

# Impute the missing in test set
- merge the training and test with connectome matrices
- Use MICE imputation to impute missing age in the training set, and missing survey responses in the test set

In [3]:
train_merged.columns

Index(['Barratt_Barratt_P1_Edu_college_edu',
       'Barratt_Barratt_P1_Edu_graduate_edu',
       'Barratt_Barratt_P1_Edu_high_school_grad',
       'Barratt_Barratt_P1_Edu_junior_middle_school',
       'Barratt_Barratt_P1_Edu_less_than_7th',
       'Barratt_Barratt_P1_Edu_partial_college',
       'Barratt_Barratt_P1_Edu_partial_high_school',
       'Barratt_Barratt_P1_Edu_unknown', 'Barratt_Barratt_P1_Occ_0.0',
       'Barratt_Barratt_P1_Occ_1.0', 'Barratt_Barratt_P1_Occ_10.0',
       'Barratt_Barratt_P1_Occ_15.0', 'Barratt_Barratt_P1_Occ_20.0',
       'Barratt_Barratt_P1_Occ_25.0', 'Barratt_Barratt_P1_Occ_30.0',
       'Barratt_Barratt_P1_Occ_35.0', 'Barratt_Barratt_P1_Occ_40.0',
       'Barratt_Barratt_P1_Occ_45.0', 'Barratt_Barratt_P1_Occ_5.0',
       'Barratt_Barratt_P1_Occ_nan', 'Barratt_Barratt_P2_Edu_college_edu',
       'Barratt_Barratt_P2_Edu_graduate_edu',
       'Barratt_Barratt_P2_Edu_high_school_grad',
       'Barratt_Barratt_P2_Edu_junior_middle_school',
       'Barratt_B

In [ ]:
# Merge set for imputation 
impute_df = pd.concat([train_merged, test_merged], ignore_index=True)

# Change participant_id to index
impute_df = impute_df.set_index('participant_id')


In [23]:
train_merged = train_merged.set_index('participant_id')
test_merged = test_merged.set_index('participant_id')

In [28]:
len(train_merged)

1213

In [29]:
len(test_merged)

304

In [5]:
impute_df.head()

,Barratt_Barratt_P1_Edu_college_edu,Barratt_Barratt_P1_Edu_graduate_edu,Barratt_Barratt_P1_Edu_high_school_grad,Barratt_Barratt_P1_Edu_junior_middle_school,Barratt_Barratt_P1_Edu_less_than_7th,Barratt_Barratt_P1_Edu_partial_college,Barratt_Barratt_P1_Edu_partial_high_school,Barratt_Barratt_P1_Edu_unknown,Barratt_Barratt_P1_Occ_0.0,Barratt_Barratt_P1_Occ_1.0,...,SDQ_SDQ_Conduct_Problems,SDQ_SDQ_Difficulties_Total,SDQ_SDQ_Emotional_Problems,SDQ_SDQ_Externalizing,SDQ_SDQ_Generating_Impact,SDQ_SDQ_Hyperactivity,SDQ_SDQ_Internalizing,SDQ_SDQ_Peer_Problems,SDQ_SDQ_Prosocial,MRI_Track_Age_at_Scan
participant_id,,,,,,,,,,,,,,,,,,,,,
00aIpNTbG5uh,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,3.0,17.0,4.0,11.0,5.0,8.0,6.0,2.0,9.0,14.274127
00fV0OyyoLfw,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,5.0,20.0,4.0,13.0,5.0,8.0,7.0,3.0,8.0,NaN
04X1eiS79T4B,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,...,3.0,24.0,7.0,10.0,10.0,7.0,14.0,7.0,7.0,13.463381
05ocQutkURd6,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,5.0,0.0,3.0,0.0,3.0,2.0,2.0,6.0,9.572553
06YUNBA9ZRLq,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,6.0,23.0,7.0,15.0,8.0,9.0,8.0,1.0,4.0,6.654574


In [14]:
# MICE

# Get column names 
colnames_imputed = impute_df.columns

# Initiate imputer 
imputer = IterativeImputer(random_state=100, max_iter=10)

imputer.fit(impute_df)

df_imputed = imputer.transform(impute_df)

df_imputed_final = pd.DataFrame(df_imputed, columns=colnames_imputed, index=impute_df.index)



In [16]:
df_imputed_final.head()

,Barratt_Barratt_P1_Edu_college_edu,Barratt_Barratt_P1_Edu_graduate_edu,Barratt_Barratt_P1_Edu_high_school_grad,Barratt_Barratt_P1_Edu_junior_middle_school,Barratt_Barratt_P1_Edu_less_than_7th,Barratt_Barratt_P1_Edu_partial_college,Barratt_Barratt_P1_Edu_partial_high_school,Barratt_Barratt_P1_Edu_unknown,Barratt_Barratt_P1_Occ_0.0,Barratt_Barratt_P1_Occ_1.0,...,SDQ_SDQ_Conduct_Problems,SDQ_SDQ_Difficulties_Total,SDQ_SDQ_Emotional_Problems,SDQ_SDQ_Externalizing,SDQ_SDQ_Generating_Impact,SDQ_SDQ_Hyperactivity,SDQ_SDQ_Internalizing,SDQ_SDQ_Peer_Problems,SDQ_SDQ_Prosocial,MRI_Track_Age_at_Scan
participant_id,,,,,,,,,,,,,,,,,,,,,
00aIpNTbG5uh,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,3.0,17.0,4.0,11.0,5.0,8.0,6.0,2.0,9.0,14.274127
00fV0OyyoLfw,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,5.0,20.0,4.0,13.0,5.0,8.0,7.0,3.0,8.0,14.082805
04X1eiS79T4B,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,...,3.0,24.0,7.0,10.0,10.0,7.0,14.0,7.0,7.0,13.463381
05ocQutkURd6,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,5.0,0.0,3.0,0.0,3.0,2.0,2.0,6.0,9.572553
06YUNBA9ZRLq,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,6.0,23.0,7.0,15.0,8.0,9.0,8.0,1.0,4.0,6.654574


## Multiple rounds of imputation testing by MICE, age accuracy check

In [17]:
mask = np.random.rand(len(df_imputed_final)) < 0.2
second_df_imputed = df_imputed_final.copy()
second_df_imputed.loc[mask, 'MRI_Track_Age_at_Scan'] = np.nan  

# Re-impute the missing values (Second Imputation)
df_reimputed = second_df_imputed.copy()
df_reimputed = pd.DataFrame(imputer.transform(second_df_imputed), columns=colnames_imputed)

# MAE
mae = mean_absolute_error(
    df_imputed_final.loc[mask, 'MRI_Track_Age_at_Scan'], 
    df_reimputed.loc[mask, 'MRI_Track_Age_at_Scan']
)

# Percentage
epsilon = 1e-5 
percentage_errors = np.abs((df_imputed_final.loc[mask, 'MRI_Track_Age_at_Scan'] - 
                             df_reimputed.loc[mask, 'MRI_Track_Age_at_Scan']) / 
                            (df_imputed_final.loc[mask, 'MRI_Track_Age_at_Scan'] + epsilon)) 



In [18]:
print("MAE:", mae)
print(f"Mean percentage error: {percentage_errors.mean()}")

MAE: 1.4853764495937383
Mean percentage error: nan


## Multiple round imputation by mean, age check


In [19]:
# Mean imputation for Age
mean_age = train_merged['MRI_Track_Age_at_Scan'].mean()

# Impute missing 
train_imputed_age_one = train_merged.copy()

train_imputed_age_one['MRI_Track_Age_at_Scan'] = train_merged['MRI_Track_Age_at_Scan'].fillna(mean_age)


In [20]:
# Mean imputation for Age
mean_age = train_merged['MRI_Track_Age_at_Scan'].mean()

# Impute missing 'MRI_Track_Age_at_Scan' with the mean
train_imputed_age_one = train_merged.copy()
train_imputed_age_one['MRI_Track_Age_at_Scan'] = train_merged['MRI_Track_Age_at_Scan'].fillna(mean_age)

# Create a mask to randomly remove 20% of values for testing
mask = np.random.rand(len(train_imputed_age_one)) < 0.2
train_imputed_age_two = train_imputed_age_one.copy()
train_imputed_age_two.loc[mask, 'MRI_Track_Age_at_Scan'] = np.nan
train_imputed_age_two['MRI_Track_Age_at_Scan'] = train_imputed_age_two['MRI_Track_Age_at_Scan'].fillna(mean_age)


# Calculate MAE for the imputation accuracy
mae = mean_absolute_error(
    train_imputed_age_one.loc[mask, 'MRI_Track_Age_at_Scan'], 
    train_imputed_age_two.loc[mask, 'MRI_Track_Age_at_Scan']
)

print(f"MAE: {mae}")


MAE: 2.127047070024165


## Split the dataset back to test and train 

In [21]:
df_imputed_final.head()

,Barratt_Barratt_P1_Edu_college_edu,Barratt_Barratt_P1_Edu_graduate_edu,Barratt_Barratt_P1_Edu_high_school_grad,Barratt_Barratt_P1_Edu_junior_middle_school,Barratt_Barratt_P1_Edu_less_than_7th,Barratt_Barratt_P1_Edu_partial_college,Barratt_Barratt_P1_Edu_partial_high_school,Barratt_Barratt_P1_Edu_unknown,Barratt_Barratt_P1_Occ_0.0,Barratt_Barratt_P1_Occ_1.0,...,SDQ_SDQ_Conduct_Problems,SDQ_SDQ_Difficulties_Total,SDQ_SDQ_Emotional_Problems,SDQ_SDQ_Externalizing,SDQ_SDQ_Generating_Impact,SDQ_SDQ_Hyperactivity,SDQ_SDQ_Internalizing,SDQ_SDQ_Peer_Problems,SDQ_SDQ_Prosocial,MRI_Track_Age_at_Scan
participant_id,,,,,,,,,,,,,,,,,,,,,
00aIpNTbG5uh,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,3.0,17.0,4.0,11.0,5.0,8.0,6.0,2.0,9.0,14.274127
00fV0OyyoLfw,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,5.0,20.0,4.0,13.0,5.0,8.0,7.0,3.0,8.0,14.082805
04X1eiS79T4B,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,...,3.0,24.0,7.0,10.0,10.0,7.0,14.0,7.0,7.0,13.463381
05ocQutkURd6,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,5.0,0.0,3.0,0.0,3.0,2.0,2.0,6.0,9.572553
06YUNBA9ZRLq,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,6.0,23.0,7.0,15.0,8.0,9.0,8.0,1.0,4.0,6.654574


In [24]:
# Split
train_imputed = df_imputed_final.loc[train_merged.index]
test_imputed = df_imputed_final.loc[test_merged.index]


In [25]:
train_imputed

,Barratt_Barratt_P1_Edu_college_edu,Barratt_Barratt_P1_Edu_graduate_edu,Barratt_Barratt_P1_Edu_high_school_grad,Barratt_Barratt_P1_Edu_junior_middle_school,Barratt_Barratt_P1_Edu_less_than_7th,Barratt_Barratt_P1_Edu_partial_college,Barratt_Barratt_P1_Edu_partial_high_school,Barratt_Barratt_P1_Edu_unknown,Barratt_Barratt_P1_Occ_0.0,Barratt_Barratt_P1_Occ_1.0,...,SDQ_SDQ_Conduct_Problems,SDQ_SDQ_Difficulties_Total,SDQ_SDQ_Emotional_Problems,SDQ_SDQ_Externalizing,SDQ_SDQ_Generating_Impact,SDQ_SDQ_Hyperactivity,SDQ_SDQ_Internalizing,SDQ_SDQ_Peer_Problems,SDQ_SDQ_Prosocial,MRI_Track_Age_at_Scan
participant_id,,,,,,,,,,,,,,,,,,,,,
00aIpNTbG5uh,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,3.0,17.0,4.0,11.0,5.0,8.0,6.0,2.0,9.0,14.274127
00fV0OyyoLfw,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,5.0,20.0,4.0,13.0,5.0,8.0,7.0,3.0,8.0,14.082805
04X1eiS79T4B,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,...,3.0,24.0,7.0,10.0,10.0,7.0,14.0,7.0,7.0,13.463381
05ocQutkURd6,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,5.0,0.0,3.0,0.0,3.0,2.0,2.0,6.0,9.572553
06YUNBA9ZRLq,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,6.0,23.0,7.0,15.0,8.0,9.0,8.0,1.0,4.0,6.654574
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
zwjJWCRzKhDz,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,5.0,20.0,5.0,13.0,5.0,8.0,7.0,2.0,8.0,11.272986
zwXD5v17Rx01,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.0,5.0,0.0,5.0,0.0,3.0,0.0,0.0,7.0,7.364020
zWzLCi3NTBTd,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.0,14.0,3.0,9.0,3.0,7.0,5.0,2.0,9.0,9.962126


In [26]:
test_imputed

,Barratt_Barratt_P1_Edu_college_edu,Barratt_Barratt_P1_Edu_graduate_edu,Barratt_Barratt_P1_Edu_high_school_grad,Barratt_Barratt_P1_Edu_junior_middle_school,Barratt_Barratt_P1_Edu_less_than_7th,Barratt_Barratt_P1_Edu_partial_college,Barratt_Barratt_P1_Edu_partial_high_school,Barratt_Barratt_P1_Edu_unknown,Barratt_Barratt_P1_Occ_0.0,Barratt_Barratt_P1_Occ_1.0,...,SDQ_SDQ_Conduct_Problems,SDQ_SDQ_Difficulties_Total,SDQ_SDQ_Emotional_Problems,SDQ_SDQ_Externalizing,SDQ_SDQ_Generating_Impact,SDQ_SDQ_Hyperactivity,SDQ_SDQ_Internalizing,SDQ_SDQ_Peer_Problems,SDQ_SDQ_Prosocial,MRI_Track_Age_at_Scan
participant_id,,,,,,,,,,,,,,,,,,,,,
Cfwaf5FX7jWK,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.0,12.0,3.0,9.0,2.0,7.0,3.0,0.0,8.0,8.992813
vhGrzmvA3Hjq,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.0,16.0,8.0,5.0,7.0,3.0,11.0,3.0,9.0,12.324093
ULliyEXjy4OV,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,7.0,1.0,6.0,1.0,5.0,1.0,0.0,9.0,7.770933
LZfeAb1xMtql,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,4.0,15.0,4.0,10.0,8.0,6.0,5.0,1.0,6.0,9.304814
EnFOUv0YK1RG,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,2.0,18.0,6.0,12.0,5.0,10.0,6.0,0.0,10.0,8.261350
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
UadZfjdEg7eG,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,1.0,18.0,7.0,7.0,5.0,6.0,11.0,4.0,7.0,7.546999
IUEHiLmQAqCi,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.0,16.0,2.0,11.0,5.0,9.0,5.0,3.0,8.0,10.531143
cRySmCadYFRO,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,11.0,4.0,4.0,4.0,3.0,7.0,3.0,10.0,7.210586


In [27]:
# Write to CSV
train_imputed.to_csv("train_cat_quant_imputed.csv")
test_imputed.to_csv("test_cat_quant_imputed.csv")